# deprecated


In [ ]:
from mainnet_launch.data_fetching.get_state_by_block import (
    get_state_by_one_block,
    build_blocks_to_use,
    get_raw_state_by_blocks,
    safe_normalize_6_with_bool_success,
    safe_normalize_with_bool_success,
)

import os
import pandas as pd
from tqdm import tqdm
import concurrent.futures as cf

from mainnet_launch.database.postgres_operations import (
    get_full_table_as_df_with_block,
    get_full_table_as_df,
    get_full_table_as_df_with_tx_hash,
)
from mainnet_launch.database.schema.full import (
    DestinationStates,
    Destinations,
    AutopoolDestinations,
    RebalanceEvents,
    RebalancePlans,
    Blocks,
)
import pandas as pd

from multicall import Call
import numpy as np
import time
import datetime
from mainnet_launch.data_fetching.defi_llama.fetch_timestamp import fetch_blocks_by_unix_timestamps_defillama
from mainnet_launch.data_fetching.internal.s3_helper import fetch_rebalance_plan_json_no_s3_client
import plotly.express as px
import plotly.io as pio


from mainnet_launch.constants import (
    AUTO_USD,
    ETH_CHAIN,
    AUTO_ETH,
    AutopoolConstants,
    AUTO_DOLA,
    BASE_USD,
    BASE_CHAIN,
    ALL_AUTOPOOLS,
    BASE_ETH,
)


from mainnet_launch.data_fetching.get_state_by_block import (
    get_state_by_one_block,
    build_blocks_to_use,
    get_raw_state_by_blocks,
    safe_normalize_6_with_bool_success,
    safe_normalize_with_bool_success,
    identity_with_bool_success,
)


from mainnet_launch.data_fetching.internal.s3_helper import fetch_rebalance_plan_json_no_s3_client

pio.templates.default = None


autopool = AUTO_ETH

destinations = get_full_table_as_df(Destinations, where_clause=Destinations.chain_id == autopool.chain.chain_id)
autopool_destinations = get_full_table_as_df(
    AutopoolDestinations, where_clause=AutopoolDestinations.autopool_vault_address == autopool.autopool_eth_addr
)
# # 2 min to fetch
destination_states = get_full_table_as_df_with_block(
    DestinationStates,
    where_clause=DestinationStates.destination_vault_address.in_(destinations["destination_vault_address"].tolist()),
)
plans = get_full_table_as_df(
    RebalancePlans, where_clause=RebalancePlans.autopool_vault_address == autopool.autopool_eth_addr
).sort_values("datetime_generated")
events = get_full_table_as_df_with_tx_hash(
    RebalanceEvents, where_clause=RebalanceEvents.autopool_vault_address == autopool.autopool_eth_addr
).sort_index()
mainnet_blocks = get_full_table_as_df(Blocks, where_clause=Blocks.chain_id == autopool.chain.chain_id).sort_values(
    "block"
)

calls = [
    Call(address, "getStats()(address)", [(address, identity_with_bool_success)])
    for address in destinations["destination_vault_address"].values
]
events.reset_index()

stats_addresses = get_state_by_one_block(calls, autopool.chain.get_block_near_top(), autopool.chain)

In [ ]:
events

,tx_hash,autopool_vault_address,chain_id,rebalance_file_path,destination_out,destination_in,quantity_out,quantity_in,safe_value_out,safe_value_in,spot_value_in,spot_value_out,spot_value_in_solver_change,block,out_fee_apr,out_base_apr,out_fee_plus_base_apr
datetime,,,,,,,,,,,,,,,,,
2024-09-17 19:42:23+00:00,0xf4b0efb2e5d69ef7b7d000b4860a9d86b76013d9a042...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1726602027_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xE382BBd32C4E202185762eA433278f4ED9E6151E,100.000000,99.961014,100.000000,99.963008,99.966261,100.000000,0.000000,20772483,None,None,None
2024-09-18 04:02:23+00:00,0xfdb95550d0720ff1f6ca2ff7331d1eb3194fe85ee8ab...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1726632022_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xE382BBd32C4E202185762eA433278f4ED9E6151E,314.500371,314.380624,314.500371,314.372549,314.387618,314.500371,0.000000,20774967,None,None,None
2024-09-18 16:22:35+00:00,0xaa013ce865573532459a55487fb809c3e93edeed05af...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1726676459_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,146.034135,144.676019,146.034135,145.966127,145.967367,146.034135,0.021905,20778643,None,None,None
2024-09-19 02:57:47+00:00,0x1e5ddbb3a2e56692c1c261b05b482298c1cd6bd6bcf3...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1726714560_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,134.705433,133.437132,134.705433,134.644712,134.648167,134.705433,0.020206,20781802,None,None,None
2024-09-19 13:26:23+00:00,0xfb4645f276bf56d5ff8fef87301516ccb8ed0841a5db...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1726752261_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xE382BBd32C4E202185762eA433278f4ED9E6151E,335.831303,335.615197,335.831303,335.660996,335.663713,335.831303,-0.042111,20784928,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-12 12:31:11+00:00,0x3980b42c8756999dcd7401c330c685027b5291261999...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1768220996_0x0A2b94F6871c1D7A32...,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,47.263918,39.475138,47.366768,47.283073,47.292483,47.326576,0.011828,24218647,None,None,None
2026-01-12 22:31:11+00:00,0xa00be293f95ab78c3b64c2ef8cd2411a3e4d0a2aa309...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1768256996_0x0A2b94F6871c1D7A32...,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,33.191028,27.731277,33.205006,33.216349,33.222772,33.227399,0.008311,24221631,None,None,None
2026-01-13 14:30:59+00:00,0x078c6ea3b5e542eb59f58a9056f69646c331620e671c...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1768314596_0x0A2b94F6871c1D7A32...,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,24.893271,20.796678,24.936594,24.909304,24.917280,24.937640,0.006235,24226402,None,None,None


In [ ]:
events

,tx_hash,autopool_vault_address,chain_id,rebalance_file_path,destination_out,destination_in,quantity_out,quantity_in,safe_value_out,safe_value_in,spot_value_in,spot_value_out,spot_value_in_solver_change,block,out_fee_apr,out_base_apr,out_fee_plus_base_apr
datetime,,,,,,,,,,,,,,,,,
2024-09-17 19:42:23+00:00,0xf4b0efb2e5d69ef7b7d000b4860a9d86b76013d9a042...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1726602027_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xE382BBd32C4E202185762eA433278f4ED9E6151E,100.000000,99.961014,100.000000,99.963008,99.966261,100.000000,0.000000,20772483,None,None,None
2024-09-18 04:02:23+00:00,0xfdb95550d0720ff1f6ca2ff7331d1eb3194fe85ee8ab...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1726632022_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xE382BBd32C4E202185762eA433278f4ED9E6151E,314.500371,314.380624,314.500371,314.372549,314.387618,314.500371,0.000000,20774967,None,None,None
2024-09-18 16:22:35+00:00,0xaa013ce865573532459a55487fb809c3e93edeed05af...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1726676459_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,146.034135,144.676019,146.034135,145.966127,145.967367,146.034135,0.021905,20778643,None,None,None
2024-09-19 02:57:47+00:00,0x1e5ddbb3a2e56692c1c261b05b482298c1cd6bd6bcf3...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1726714560_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,134.705433,133.437132,134.705433,134.644712,134.648167,134.705433,0.020206,20781802,None,None,None
2024-09-19 13:26:23+00:00,0xfb4645f276bf56d5ff8fef87301516ccb8ed0841a5db...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1726752261_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xE382BBd32C4E202185762eA433278f4ED9E6151E,335.831303,335.615197,335.831303,335.660996,335.663713,335.831303,-0.042111,20784928,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-12 12:31:11+00:00,0x3980b42c8756999dcd7401c330c685027b5291261999...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1768220996_0x0A2b94F6871c1D7A32...,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,47.263918,39.475138,47.366768,47.283073,47.292483,47.326576,0.011828,24218647,None,None,None
2026-01-12 22:31:11+00:00,0xa00be293f95ab78c3b64c2ef8cd2411a3e4d0a2aa309...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1768256996_0x0A2b94F6871c1D7A32...,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,33.191028,27.731277,33.205006,33.216349,33.222772,33.227399,0.008311,24221631,None,None,None
2026-01-13 14:30:59+00:00,0x078c6ea3b5e542eb59f58a9056f69646c331620e671c...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1768314596_0x0A2b94F6871c1D7A32...,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,24.893271,20.796678,24.936594,24.909304,24.917280,24.937640,0.006235,24226402,None,None,None


In [ ]:
apr_cols = [
    "out_incentive_apr",
    "out_fee_apr",
    "out_base_apr",
    "out_points_apr",
    "out_fee_plus_base_apr",
    "in_incentive_apr",
    "in_fee_apr",
    "in_base_apr",
    "in_points_apr",
    "in_fee_plus_base_apr",
]

id_cols = [
    "rebalance_plan_key",
    "block",
    "block_datetime",
    "destination_out_name",
    "destination_in_name",
    "destination_out",
    "destination_in",
]

intersting_cols = [*id_cols, *apr_cols]


def _add_apr_info_to_executed_plans(executed_plans, some_destination_states, autopool):
    key_to_incentive_apr = some_destination_states.set_index(["rebalance_plan_key", "destination_vault_address"])[
        "incentive_apr"
    ].to_dict()
    key_to_fee_apr = some_destination_states.set_index(["rebalance_plan_key", "destination_vault_address"])[
        "fee_apr"
    ].to_dict()
    key_to_base_apr = some_destination_states.set_index(["rebalance_plan_key", "destination_vault_address"])[
        "base_apr"
    ].to_dict()
    key_to_points_apr = some_destination_states.set_index(["rebalance_plan_key", "destination_vault_address"])[
        "points_apr"
    ].to_dict()
    key_to_fee_plus_base_apr = some_destination_states.set_index(["rebalance_plan_key", "destination_vault_address"])[
        "fee_plus_base_apr"
    ].to_dict()

    executed_plans["out_incentive_apr"] = executed_plans.apply(
        lambda row: key_to_incentive_apr.get((row["rebalance_plan_key"], row["destination_out"]), np.nan), axis=1
    )
    executed_plans["out_fee_apr"] = executed_plans.apply(
        lambda row: key_to_fee_apr.get((row["rebalance_plan_key"], row["destination_out"]), np.nan), axis=1
    )
    executed_plans["out_base_apr"] = executed_plans.apply(
        lambda row: key_to_base_apr.get((row["rebalance_plan_key"], row["destination_out"]), np.nan), axis=1
    )
    executed_plans["out_points_apr"] = executed_plans.apply(
        lambda row: key_to_points_apr.get((row["rebalance_plan_key"], row["destination_out"]), np.nan), axis=1
    )
    executed_plans["out_fee_plus_base_apr"] = executed_plans.apply(
        lambda row: key_to_fee_plus_base_apr.get((row["rebalance_plan_key"], row["destination_out"]), np.nan), axis=1
    )
    executed_plans["in_incentive_apr"] = executed_plans.apply(
        lambda row: key_to_incentive_apr.get((row["rebalance_plan_key"], row["destination_in"]), np.nan), axis=1
    )
    executed_plans["in_fee_apr"] = executed_plans.apply(
        lambda row: key_to_fee_apr.get((row["rebalance_plan_key"], row["destination_in"]), np.nan), axis=1
    )
    executed_plans["in_base_apr"] = executed_plans.apply(
        lambda row: key_to_base_apr.get((row["rebalance_plan_key"], row["destination_in"]), np.nan), axis=1
    )
    executed_plans["in_points_apr"] = executed_plans.apply(
        lambda row: key_to_points_apr.get((row["rebalance_plan_key"], row["destination_in"]), np.nan), axis=1
    )
    executed_plans["in_fee_plus_base_apr"] = executed_plans.apply(
        lambda row: key_to_fee_plus_base_apr.get((row["rebalance_plan_key"], row["destination_in"]), np.nan), axis=1
    )
    # s
    executed_plans.loc[
        executed_plans["destination_out"] == autopool.autopool_eth_addr,
        ["out_incentive_apr", "out_fee_apr", "out_base_apr", "out_points_apr", "out_fee_plus_base_apr"],
    ] = 0
    executed_plans.loc[
        executed_plans["destination_in"] == autopool.autopool_eth_addr,
        ["in_incentive_apr", "in_fee_apr", "in_base_apr", "in_points_apr", "in_fee_plus_base_apr"],
    ] = 0

    # for readability, multiply all APR columns by 100 to convert to percentage
    for col in apr_cols:
        executed_plans[col] = executed_plans[col] * 100

    return executed_plans


destinations["stats"] = destinations["destination_vault_address"].map(stats_addresses)

some_destination_states = destination_states[~destination_states["rebalance_plan_key"].isna()].copy()
executed_plans = (
    plans[plans["file_name"].isin(events["rebalance_file_path"])]
    .copy()
    .rename(columns={"file_name": "rebalance_plan_key"})
    .copy()
)

executed_plans["destination_out_name"] = executed_plans["destination_out"].map(
    destinations.set_index("destination_vault_address")["underlying_name"]
)
executed_plans["destination_in_name"] = executed_plans["destination_in"].map(
    destinations.set_index("destination_vault_address")["underlying_name"]
)
executed_plans["block"] = executed_plans["rebalance_plan_key"].map(
    events.set_index("rebalance_file_path")["block"].to_dict()
)
executed_plans["block_datetime"] = executed_plans["rebalance_plan_key"].map(
    events.reset_index().set_index("rebalance_file_path")["datetime"].to_dict()
)

executed_plans = _add_apr_info_to_executed_plans(executed_plans, some_destination_states, autopool)
executed_plans[intersting_cols]

,rebalance_plan_key,block,block_datetime,destination_out_name,destination_in_name,destination_out,destination_in,out_incentive_apr,out_fee_apr,out_base_apr,out_points_apr,out_fee_plus_base_apr,in_incentive_apr,in_fee_apr,in_base_apr,in_points_apr,in_fee_plus_base_apr
1072,rebalance_plan_1726602027_0x0A2b94F6871c1D7A32...,20772483,2024-09-17 19:42:23+00:00,autoETH,pxETH/wETH,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0.000000,0.0,0.0,0.0,0.000000,11.474543,NaN,NaN,NaN,0.187149
1075,rebalance_plan_1726632022_0x0A2b94F6871c1D7A32...,20774967,2024-09-18 04:02:23+00:00,autoETH,pxETH/wETH,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0.000000,0.0,0.0,0.0,0.000000,10.953403,NaN,NaN,NaN,0.190564
1064,rebalance_plan_1726676459_0x0A2b94F6871c1D7A32...,20778643,2024-09-18 16:22:35+00:00,autoETH,pxETH/stETH,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,0.000000,0.0,0.0,0.0,0.000000,8.284285,NaN,NaN,NaN,1.578220
1512,rebalance_plan_1726714560_0x0A2b94F6871c1D7A32...,20781802,2024-09-19 02:57:47+00:00,autoETH,pxETH/stETH,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,0.000000,0.0,0.0,0.0,0.000000,8.359403,NaN,NaN,NaN,1.569256
574,rebalance_plan_1726752261_0x0A2b94F6871c1D7A32...,20784928,2024-09-19 13:26:23+00:00,autoETH,pxETH/wETH,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0.000000,0.0,0.0,0.0,0.000000,10.010023,NaN,NaN,NaN,0.155159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,rebalance_plan_1768084196_0x0A2b94F6871c1D7A32...,24207300,2026-01-10 22:31:11+00:00,pxETH/wETH,Instadapp ETH v2,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0.246208,NaN,NaN,NaN,0.574978,0.000000,NaN,NaN,NaN,5.122990
955,rebalance_plan_1768220996_0x0A2b94F6871c1D7A32...,24218647,2026-01-12 12:31:11+00:00,pxETH/wETH,Instadapp ETH v2,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0.000000,NaN,NaN,NaN,0.875333,0.000000,NaN,NaN,NaN,4.991118
199,rebalance_plan_1768256996_0x0A2b94F6871c1D7A32...,24221631,2026-01-12 22:31:11+00:00,pxETH/wETH,Instadapp ETH v2,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0.000000,NaN,NaN,NaN,1.107568,0.000000,NaN,NaN,NaN,4.991118
298,rebalance_plan_1768314596_0x0A2b94F6871c1D7A32...,24226402,2026-01-13 14:30:59+00:00,pxETH/wETH,Instadapp ETH v2,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0.000000,NaN,NaN,NaN,1.272576,0.000000,NaN,NaN,NaN,5.000692


In [ ]:
# need a generally smart way to get the virtual price at time T

In [ ]:
executed_plans.columns

Index(['rebalance_plan_key', 'datetime_generated', 'autopool_vault_address',
       'chain_id', 'solver_address', 'rebalance_type', 'destination_out',
       'token_out', 'destination_in', 'token_in', 'move_name', 'amount_out',
       'amount_out_safe_value', 'min_amount_in', 'min_amount_in_safe_value',
       'amount_out_spot_value', 'out_dest_apr', 'min_amount_in_spot_value',
       'in_dest_apr', 'in_dest_adj_apr', 'apr_delta', 'swap_offset_period',
       'num_candidate_destinations', 'candidate_destinations_rank',
       'destination_out_name', 'destination_in_name', 'out_incentive_apr',
       'out_fee_apr', 'out_base_apr', 'out_points_apr',
       'out_fee_plus_base_apr', 'in_incentive_apr', 'in_fee_apr',
       'in_base_apr', 'in_points_apr', 'in_fee_plus_base_apr'],
      dtype='object')

In [ ]:
break

,rebalance_plan_key,destination_out_name,destination_in_name,out_incentive_apr,out_fee_apr,out_base_apr,out_points_apr,out_fee_plus_base_apr,in_incentive_apr,in_fee_apr,in_base_apr,in_points_apr,in_fee_plus_base_apr
1072,rebalance_plan_1726602027_0x0A2b94F6871c1D7A32...,autoETH,pxETH/wETH,0.000000,0.0,0.0,0.0,0.000000,11.474543,NaN,NaN,NaN,0.187149
1075,rebalance_plan_1726632022_0x0A2b94F6871c1D7A32...,autoETH,pxETH/wETH,0.000000,0.0,0.0,0.0,0.000000,10.953403,NaN,NaN,NaN,0.190564
1064,rebalance_plan_1726676459_0x0A2b94F6871c1D7A32...,autoETH,pxETH/stETH,0.000000,0.0,0.0,0.0,0.000000,8.284285,NaN,NaN,NaN,1.578220
1512,rebalance_plan_1726714560_0x0A2b94F6871c1D7A32...,autoETH,pxETH/stETH,0.000000,0.0,0.0,0.0,0.000000,8.359403,NaN,NaN,NaN,1.569256
574,rebalance_plan_1726752261_0x0A2b94F6871c1D7A32...,autoETH,pxETH/wETH,0.000000,0.0,0.0,0.0,0.000000,10.010023,NaN,NaN,NaN,0.155159
...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,rebalance_plan_1768084196_0x0A2b94F6871c1D7A32...,pxETH/wETH,Instadapp ETH v2,0.246208,NaN,NaN,NaN,0.574978,0.000000,NaN,NaN,NaN,5.122990
955,rebalance_plan_1768220996_0x0A2b94F6871c1D7A32...,pxETH/wETH,Instadapp ETH v2,0.000000,NaN,NaN,NaN,0.875333,0.000000,NaN,NaN,NaN,4.991118
199,rebalance_plan_1768256996_0x0A2b94F6871c1D7A32...,pxETH/wETH,Instadapp ETH v2,0.000000,NaN,NaN,NaN,1.107568,0.000000,NaN,NaN,NaN,4.991118
298,rebalance_plan_1768314596_0x0A2b94F6871c1D7A32...,pxETH/wETH,Instadapp ETH v2,0.000000,NaN,NaN,NaN,1.272576,0.000000,NaN,NaN,NaN,5.000692


In [ ]:
# # Get all plan timestamps
# plan_timestamps = plans['datetime_generated'].tolist()

# # For each plan timestamp, check if there are destination states within 2 hours
# results = []
# for plan_timestamp in plan_timestamps:
#     time_window = datetime.timedelta(hours=12)
#     states_in_window = destination_states[
#         (destination_states.index >= plan_timestamp - time_window) &
#         (destination_states.index <= plan_timestamp + time_window)
#     ]

#     results.append({
#         'plan_timestamp': plan_timestamp,
#         'num_dest_states': len(states_in_window),
#         'has_states': len(states_in_window) > 0,
#         'unique_destinations': states_in_window['destination_vault_address'].nunique() if len(states_in_window) > 0 else 0
#     })

# results_df = pd.DataFrame(results)
# print(results_df.has_states.value_counts(normalize=True))
# px.line(results_df, x='plan_timestamp', y='unique_destinations',
#     title='Unique Destinations Count Over Plan Timestamp',
#     labels={'plan_timestamp': 'Plan Timestamp', 'unique_destinations': 'Unique Destinations'})


# # I want to join the plan to the destiantions states that is closest within 12 hours,
# # Merge destination states with the nearest plan timestamp within 12 hours
# # Use merge_asof for nearest timestamp matching
# plans_for_merge = plans[['datetime_generated']].copy()
# plans_for_merge = plans_for_merge.sort_values('datetime_generated')

# # Prepare destination_states with sorted index
# dest_states_sorted = destination_states.sort_index()

# # Use merge_asof to find nearest plan timestamp for each destination state
# merged_states = pd.merge_asof(
#     dest_states_sorted.reset_index(),
#     plans_for_merge.reset_index(),
#     left_on='datetime',
#     right_on='datetime_generated',
#     direction='nearest',
#     tolerance=pd.Timedelta(hours=12)
# )

# # Set datetime back as index
# merged_states = merged_states.set_index('datetime')

# print(f"Total destination states: {len(destination_states)}")
# print(f"States matched to plans: {merged_states['datetime_generated'].notna().sum()}")
# print(f"Match rate: {merged_states['datetime_generated'].notna().mean():.1%}")

# merged_states['name'] = merged_states['destination_vault_address'].map(destinations.set_index('destination_vault_address')['name'])
# merged_states['exchange_name'] = merged_states['destination_vault_address'].map(destinations.set_index('destination_vault_address')['exchange_name'])
# merged_states

In [ ]:
break

In [ ]:
# Build a dict of (destination_vault_address, datetime) -> fee, base and fee + base apr
dest_state_apr_dict = {
    (row["destination_vault_address"], idx): {
        "fee_apr": row["fee_apr"],
        "base_apr": row["base_apr"],
        "fee_plus_base_apr": row["fee_plus_base_apr"],
        "datetime": idx,
    }
    for idx, row in destination_states.iterrows()
}

# Display the first value in the dictionary
first_key = next(iter(dest_state_apr_dict))
print(f"First key: {first_key}")
print(f"First value: {dest_state_apr_dict[first_key]}")

# Map APR data from dest_state_apr_dict to destination_out
events["out_fee_apr"] = events.apply(
    lambda row: dest_state_apr_dict.get((row["destination_out"], row.name), {}).get("fee_apr"), axis=1
)

events["out_base_apr"] = events.apply(
    lambda row: dest_state_apr_dict.get((row["destination_out"], row.name), {}).get("base_apr"), axis=1
)

events["out_fee_plus_base_apr"] = events.apply(
    lambda row: dest_state_apr_dict.get((row["destination_out"], row.name), {}).get("fee_plus_base_apr"), axis=1
)

events[["out_fee_apr", "out_base_apr", "out_fee_plus_base_apr"]].isna().value_counts()

First key: ('0x232d4755B2d88895472AfBD7F0C6D0acb3b65D04', Timestamp('2026-02-06 06:39:47+0000', tz='UTC'))
First value: {'fee_apr': nan, 'base_apr': nan, 'fee_plus_base_apr': 0.0, 'datetime': Timestamp('2026-02-06 06:39:47+0000', tz='UTC')}


out_fee_apr  out_base_apr  out_fee_plus_base_apr
True         True          True                     651
Name: count, dtype: int64

In [ ]:
# Map APR data from dest_state_apr_dict to destination_out
events["out_fee_apr"] = events.apply(
    lambda row: dest_state_apr_dict.get((row["destination_out"], row.name), {}).get("fee_apr"), axis=1
)

events["out_base_apr"] = events.apply(
    lambda row: dest_state_apr_dict.get((row["destination_out"], row.name), {}).get("base_apr"), axis=1
)

events["out_fee_plus_base_apr"] = events.apply(
    lambda row: dest_state_apr_dict.get((row["destination_out"], row.name), {}).get("fee_plus_base_apr"), axis=1
)

events[["out_fee_apr", "out_base_apr", "out_fee_plus_base_apr"]].isna().value_counts()

out_fee_apr  out_base_apr  out_fee_plus_base_apr
True         True          True                     651
Name: count, dtype: int64

In [ ]:
executed_plans

,file_name,datetime_generated,autopool_vault_address,chain_id,solver_address,rebalance_type,destination_out,token_out,destination_in,token_in,...,min_amount_in_safe_value,amount_out_spot_value,out_dest_apr,min_amount_in_spot_value,in_dest_apr,in_dest_adj_apr,apr_delta,swap_offset_period,num_candidate_destinations,candidate_destinations_rank
1072,rebalance_plan_1726602027_0x0A2b94F6871c1D7A32...,2024-09-17 19:40:27+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,...,99.976617,None,NaN,None,0.104575,0.100450,0.100450,50.0,13.0,0.0
1075,rebalance_plan_1726632022_0x0A2b94F6871c1D7A32...,2024-09-18 04:00:22+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,...,314.405825,None,NaN,None,0.099885,0.088960,0.088960,50.0,13.0,0.0
1064,rebalance_plan_1726676459_0x0A2b94F6871c1D7A32...,2024-09-18 16:20:59+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,...,145.995352,None,NaN,None,0.089654,0.086579,0.086579,50.0,13.0,0.0
1512,rebalance_plan_1726714560_0x0A2b94F6871c1D7A32...,2024-09-19 02:56:00+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,...,134.671695,None,NaN,None,0.090340,0.087468,0.087468,50.0,13.0,0.0
574,rebalance_plan_1726752261_0x0A2b94F6871c1D7A32...,2024-09-19 13:24:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,...,335.722532,None,NaN,None,0.091166,0.081482,0.081482,50.0,13.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,rebalance_plan_1768084196_0x0A2b94F6871c1D7A32...,2026-01-10 22:29:56+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,Pool2Pool,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78,...,63.074479,None,0.007966,None,0.051203,0.051203,0.043237,14.0,1.0,0.0
955,rebalance_plan_1768220996_0x0A2b94F6871c1D7A32...,2026-01-12 12:29:56+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,Pool2Pool,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78,...,47.320930,None,0.008753,None,0.049761,0.049761,0.041008,14.0,1.0,0.0
199,rebalance_plan_1768256996_0x0A2b94F6871c1D7A32...,2026-01-12 22:29:56+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,Pool2Pool,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78,...,33.242944,None,0.011076,None,0.049761,0.049761,0.038686,14.0,1.0,0.0
298,rebalance_plan_1768314596_0x0A2b94F6871c1D7A32...,2026-01-13 14:29:56+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,Pool2Pool,0x43D99d04985ef2231f7d9b

In [ ]:
states = destination_states.reset_index()
out_states = states[
    [
        "rebalance_plan_key",
        "destination_vault_address",
        "fee_apr",
        "base_apr",
        "fee_plus_base_apr",
        "points_apr",
        "incentive_apr",
    ]
].copy()
out_states = out_states.rename(
    columns={
        "rebalance_plan_key": "file_name",
        "destination_vault_address": "destination_out",
        "fee_apr": "fee_apr_out",
        "base_apr": "base_apr_out",
        "fee_plus_base_apr": "fee_plus_base_apr_out",
        "points_apr": "points_apr_out",
        "incentive_apr": "incentive_apr_out",
    }
)

df = pd.merge(executed_plans, out_states, on="file_name", how="left")
df

,file_name,datetime_generated,autopool_vault_address,chain_id,solver_address,rebalance_type,destination_out_x,token_out,destination_in,token_in,...,apr_delta,swap_offset_period,num_candidate_destinations,candidate_destinations_rank,destination_out_y,fee_apr_out,base_apr_out,fee_plus_base_apr_out,points_apr_out,incentive_apr_out
0,rebalance_plan_1726602027_0x0A2b94F6871c1D7A32...,2024-09-17 19:40:27+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,...,0.100450,50.0,13.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,rebalance_plan_1726632022_0x0A2b94F6871c1D7A32...,2024-09-18 04:00:22+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,...,0.088960,50.0,13.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,rebalance_plan_1726676459_0x0A2b94F6871c1D7A32...,2024-09-18 16:20:59+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,...,0.086579,50.0,13.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,rebalance_plan_1726714560_0x0A2b94F6871c1D7A32...,2024-09-19 02:56:00+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,...,0.087468,50.0,13.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,rebalance_plan_1726752261_0x0A2b94F6871c1D7A32...,2024-09-19 13:24:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,...,0.081482,50.0,13.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,rebalance_plan_1768084196_0x0A2b94F6871c1D7A32...,2026-01-10 22:29:56+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,Pool2Pool,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78,...,0.043237,14.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
638,rebalance_plan_1768220996_0x0A2b94F6871c1D7A32...,2026-01-12 12:29:56+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,Pool2Pool,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78,...,0.041008,14.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
639,rebalance_plan_1768256996_0x0A2b94F6871c1D7A32...,2026-01-12 22:29:56+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,Pool2Pool,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78,...,0.038686,14.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
640,rebalance_plan_1768314596_0x0A2b94F6871c1D7A32...,2026-01-13 14:29:56+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,Pool2Pool,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78,...,0.037035,14.0,1.0,0.0,NaN,NaN,NaN,NaN,

In [ ]:
limited_plans = executed_plans.tail(5)
limited_plans

,file_name,datetime_generated,autopool_vault_address,chain_id,solver_address,rebalance_type,destination_out,token_out,destination_in,token_in,...,min_amount_in_safe_value,amount_out_spot_value,out_dest_apr,min_amount_in_spot_value,in_dest_apr,in_dest_adj_apr,apr_delta,swap_offset_period,num_candidate_destinations,candidate_destinations_rank
719,rebalance_plan_1768084196_0x0A2b94F6871c1D7A32...,2026-01-10 22:29:56+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,Pool2Pool,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78,...,63.074479,None,0.007966,None,0.051203,0.051203,0.043237,14.0,1.0,0.0
955,rebalance_plan_1768220996_0x0A2b94F6871c1D7A32...,2026-01-12 12:29:56+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,Pool2Pool,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78,...,47.320930,None,0.008753,None,0.049761,0.049761,0.041008,14.0,1.0,0.0
199,rebalance_plan_1768256996_0x0A2b94F6871c1D7A32...,2026-01-12 22:29:56+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,Pool2Pool,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78,...,33.242944,None,0.011076,None,0.049761,0.049761,0.038686,14.0,1.0,0.0
298,rebalance_plan_1768314596_0x0A2b94F6871c1D7A32...,2026-01-13 14:29:56+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,Pool2Pool,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78,...,24.929247,None,0.012726,None,0.049761,0.049761,0.037035,14.0,1.0,0.0
640,rebalance_plan_1769365797_0x0A2b94F6871c1D7A32...,2026-01-25 18:29:57+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78,...,50.260875,None,NaN,None,0.047689,0.047689,0.047689,14.0,1.0,0.0


In [ ]:
destination_states[destination_states["destination_vault_address"] == "0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56"]

,destination_vault_address,block,chain_id,incentive_apr,fee_apr,base_apr,points_apr,fee_plus_base_apr,total_apr_in,total_apr_out,underlying_token_total_supply,safe_total_supply,lp_token_spot_price,lp_token_safe_price,from_rebalance_plan,rebalance_plan_timestamp,rebalance_plan_key
datetime,,,,,,,,,,,,,,,,,
2026-02-04 23:59:47+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,24386913,1,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,False,NaN,None
2026-02-03 23:59:47+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,24379761,1,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,False,NaN,None
2026-02-02 23:59:47+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,24372612,1,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0,1.0,False,NaN,None
2026-02-01 23:59:47+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,24365452,1,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0,1.0,False,NaN,None
2026-01-31 23:59:47+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,24358291,1,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,False,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-11 23:57:11+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,20730797,1,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0,1.0,False,NaN,None
2024-09-11 20:00:11+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,20729615,1,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,False,NaN,None
2024-09-11 19:59:59+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,20729614,1,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,False,NaN,None


In [ ]:
df = pd.merge(
    merged_states,
    events,
    left_on=["destination_vault_address", "rebalance_plan_key"],
    right_on=["destination_vault_address", "rebalance_file_path"],
    how="left",
)

KeyError: 'destination_vault_address'

In [ ]:
import pandas as pd

TOL = pd.Timedelta("12H")

# --- prep destination_states (right) ---
ds = destination_states.copy()
ds["state_dt"] = pd.to_datetime(ds.index, utc=True)

ds = ds.reset_index(drop=True).rename(columns={"destination_vault_address": "destination_out"})[
    ["destination_out", "state_dt", "fee_apr", "base_apr", "fee_plus_base_apr"]
]

# --- prep events (left) ---
events2 = events.copy()
events2["event_dt"] = pd.to_datetime(events2.index, utc=True)
events2 = events2.reset_index(drop=False).rename(columns={"index": "orig_index"})

out = []

for dest, e_sub in events2.groupby("destination_out", sort=False):
    e_sub = e_sub.sort_values("event_dt")

    ds_sub = ds[ds["destination_out"] == dest].sort_values("state_dt")
    if ds_sub.empty:
        # no states for this destination -> keep NaNs for apr columns
        e_sub["fee_apr"] = pd.NA
        e_sub["base_apr"] = pd.NA
        e_sub["fee_plus_base_apr"] = pd.NA
        out.append(e_sub)
        continue

    m = pd.merge_asof(
        e_sub,
        ds_sub,
        left_on="event_dt",
        right_on="state_dt",
        direction="nearest",
        tolerance=TOL,
        suffixes=("", "_state"),
    )
    out.append(m)

merged = pd.concat(out, ignore_index=True).sort_values("orig_index")

events["out_fee_apr"] = merged["fee_apr"].to_numpy()
events["out_base_apr"] = merged["base_apr"].to_numpy()
events["out_fee_plus_base_apr"] = merged["fee_plus_base_apr"].to_numpy()

events[["out_fee_apr", "out_base_apr", "out_fee_plus_base_apr"]].isna().value_counts()

/var/folders/7y/zqhjt5cx12n1p79c5pfxszvm0000gn/T/ipykernel_43814/3323916959.py:3: FutureWarning:

'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.



out_fee_apr  out_base_apr  out_fee_plus_base_apr
True         True          True                     651
Name: count, dtype: int64

In [ ]:
plans.columns

Index(['file_name', 'datetime_generated', 'autopool_vault_address', 'chain_id',
       'solver_address', 'rebalance_type', 'destination_out', 'token_out',
       'destination_in', 'token_in', 'move_name', 'amount_out',
       'amount_out_safe_value', 'min_amount_in', 'min_amount_in_safe_value',
       'amount_out_spot_value', 'out_dest_apr', 'min_amount_in_spot_value',
       'in_dest_apr', 'in_dest_adj_apr', 'apr_delta', 'swap_offset_period',
       'num_candidate_destinations', 'candidate_destinations_rank'],
      dtype='object')

In [ ]:
events.columns

Index(['tx_hash', 'autopool_vault_address', 'chain_id', 'rebalance_file_path',
       'destination_out', 'destination_in', 'quantity_out', 'quantity_in',
       'safe_value_out', 'safe_value_in', 'spot_value_in', 'spot_value_out',
       'spot_value_in_solver_change', 'block'],
      dtype='object')

,destination_vault_address,block,chain_id,incentive_apr,fee_apr,base_apr,points_apr,fee_plus_base_apr,total_apr_in,total_apr_out,underlying_token_total_supply,safe_total_supply,lp_token_spot_price,lp_token_safe_price,from_rebalance_plan,rebalance_plan_timestamp,rebalance_plan_key,index,datetime_generated
datetime,,,,,,,,,,,,,,,,,,,
2024-09-10 21:31:11+00:00,0x40219bBda953ca811d2D0168Dc806a96b84791d9,20722910,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.019780,1.019947,False,NaN,None,NaN,NaT
2024-09-10 21:31:11+00:00,0x5Ff56531f92f1e3f2bE616760F05e35caE630D91,20722910,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.011948,1.012288,False,NaN,None,NaN,NaT
2024-09-10 21:31:11+00:00,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,20722910,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.013252,1.013204,False,NaN,None,NaN,NaT
2024-09-10 21:31:11+00:00,0xaD70ed6A904d38acD0BbE2D922279149cFEDb190,20722910,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.999449,0.998856,False,NaN,None,NaN,NaT
2024-09-10 21:31:11+00:00,0x3F55eedDe51504E6Ed0ec30E8289b4Da11EdB7F9,20722910,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.028141,1.028136,False,NaN,None,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-02-05 03:39:35+00:00,0x408b6A3E2Daf288864968454AAe786a2A042Df36,24388006,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.170972e+02,NaN,1.000000,1.000000,True,1.770263e+09,rebalance_plan_1770262777_0x408b6A3E2Daf288864...,1768.0,2026-02-05 02:29:56+00:00
2026-02-05 03:39:35+00:00,0x232d4755B2d88895472AfBD7F0C6D0acb3b65D04,24388006,1,0.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,9.858481e+10,NaN,1.105939,1.105968,True,1.770263e+09,rebalance_plan_1770262777_0x408b6A3E2Daf288864...,1768.0,2026-02-05 02:29:56+00:00
2026-02-05 03:39:35+00:00,0x4b1eA6f0E97cB0e859b1521a4548FBd0a94e6a63,24388006,1,0.0,NaN,NaN,NaN,0.414180,0.414180,0.414180,1.317461e+10,NaN,1.036704,1.036730,True,1.770263e+09,rebalance_plan_1770262777_0x408b6A3E2Daf288864...,1768.0,2026-02-05 02:29:56+00:00


In [ ]:
# can we cheat somehow and get it from another source? maybe infer it? can we exclude it

generated_sod[["name", "incentiveAPR", "totalAprIn", "totalAprOut"]] * 100

,name,incentiveAPR,totalAprIn,totalAprOut
0,Tokemak-Wrapped Ether-Balancer osETH-waWETHTok...,3.475104,5.057464,5.112373
1,Tokemak-Wrapped Ether-Curve.fi Factory Pool: E...,1.395892,3.154526,3.197412
2,Tokemak-Wrapped Ether-Instadapp ETH v2Tokemak-...,0.000000,4.565724,4.608338
3,Tokemak-Wrapped Ether-osETH/rETHTokemak-Wrappe...,0.000000,2.628902,2.676111
4,Tokemak-Wrapped Ether-Balancer Surge Fluid wst...,0.156624,2.519665,2.539598
5,Tokemak-Wrapped Ether-weETH/WETH-ngTokemak-Wra...,0.525114,4.885389,4.937416


In [ ]:
generated_sod[["name", "safePrice", "spotPrice"]]

,name,safePrice,spotPrice
0,Tokemak-Wrapped Ether-Balancer osETH-waWETH,1.024101,1.024137
1,Tokemak-Wrapped Ether-Curve.fi Factory Pool: E...,1.054827,1.054865
2,Tokemak-Wrapped Ether-Instadapp ETH v2,1.200566,1.200668
3,Tokemak-Wrapped Ether-osETH/rETH,1.064043,1.064048
4,Tokemak-Wrapped Ether-Balancer Surge Fluid wst...,1.021523,1.021531
5,Tokemak-Wrapped Ether-weETH/WETH-ng,1.065482,1.065471


In [ ]:
latest_generated_plan

{'timestamp': 1770258596,
 'sodOnly': False,
 'chainId': '1',
 'solverAddress': '0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D',
 'poolAddress': '0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56',
 'destinationOut': '0x9b163e15121816BE53F8d5c85fbefD6e6D9bebCd',
 'tokenOut': '0x6b31a94029fd7840d780191B6D63Fa0D269bd883',
 'amountOut': '97892300686742192128',
 'amountOutETH': '99999240879959343104',
 'destinationIn': '0x87E25fFe5C3A2720Cd43F5Eb8ec41Ac0CE699D07',
 'tokenIn': '0x57c23c58B1D8C3292c15BEcF07c62C5c52457A42',
 'minAmountIn': '97532342833066917888',
 'minAmountInETH': '99962949541446221824',
 'steps': [{'stepType': 'removeLiquidity',
   'poolType': 'balV3',
   'poolAddress': '0x6b31a94029fd7840d780191B6D63Fa0D269bd883',
   'lpTokenAddress': '0x6b31a94029fd7840d780191B6D63Fa0D269bd883',
   'lpAmountOut': '97892300686742192128',
   'tokens': ['0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0',
    '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'],
   'minTokenAmounts': ['60820721542475120640', '39188

In [ ]:
latest_auto_eth_plan = events.iloc[-1]["rebalance_file_path"]
latest_executed_plan = fetch_rebalance_plan_json_no_s3_client(latest_auto_eth_plan, autopool)
executed_sod = pd.json_normalize(latest_executed_plan["sod"]["destStates"])

latest_plan_generated = plans.iloc[-1]["file_name"]
latest_generated_plan = fetch_rebalance_plan_json_no_s3_client(latest_plan_generated, autopool)
generated_timestamp = datetime.datetime.fromtimestamp(latest_generated_plan["timestamp"], tz=datetime.timezone.utc)
print(f"Generated timestamp: {generated_timestamp}")
generated_sod = pd.json_normalize(latest_generated_plan["sod"]["destStates"])